In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir("/home/reffert/DeepAR_InfluenzaForecast")
from PythonFiles.model import model,preprocessing
from PythonFiles.Configuration import Configuration
config = Configuration()
population_df = pd.read_csv("population.csv", sep=",")
population_df.columns = population_df.columns.str.replace("31.12.","")
for i in range(2000,2022):
    population_df[f"{i}"] = population_df[f"{i}"].replace("-","0").astype(int)
influenza_df = pd.read_csv("Influenza.csv", sep=",")
geo_df = pd.read_csv("AdjacentMatrix.csv", sep=",")

/tmp/ipykernel_3344138/3856036535.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  population_df.columns = population_df.columns.str.replace("31.12.","")


                          Stichtag 31.12.2005 Unnamed: 2 31.12.2006  \
0                       Geschlecht   männlich   weiblich   männlich   
1                           Berlin    1659643    1735546    1665753   
2                        01  Mitte     164744     158000     167398   
3     02  Friedrichshain-Kreuzberg     133684     128127     135838   
4                       03  Pankow     175472     178438     177509   
5   04  Charlottenburg-Wilmersdorf     148409     166671     148892   
6                      05  Spandau     108911     116131     108644   
7          06  Steglitz-Zehlendorf     133180     155490     133137   
8         07  Tempelhof-Schöneberg     159841     173322     159774   
9                     08  Neukölln     151458     154442     151567   
10            09  Treptow-Köpenick     114717     120619     115101   
11          10 Marzahn-Hellersdorf     123905     126508     123525   
12                  11 Lichtenberg     128405     130489     128247   
13    

## Berlin Population Data

In [37]:
berlin_data = pd.read_csv("Berlin.csv", sep=",")
#print(berlin_data)
berlin_data.drop(berlin_data.loc[berlin_data["2005"].str.contains("männlich")].index, inplace=True)
#print(berlin_data.columns)
years = ['2005', '2005.1', '2006', '2006.1', '2007', '2007.1','2008', '2008.1', '2009', '2009.1', '2010', '2010.1', '2011',
         '2011.1','2012', '2012.1', '2013', '2013.1', '2014', '2014.1', '2015', '2015.1','2016', '2016.1', '2017', '2017.1',
         '2018', '2018.1']
berlin_data.loc[:,years] = berlin_data[years].replace("-","0").astype(int)
for i in range(0,28,2):
    year_m = years[i]
    year_w = years[i+1]
    berlin_data[year_m] = berlin_data[year_m] + berlin_data[year_w]
    berlin_data.drop(columns=[year_w], inplace=True)
    
#remove the numbers from the locations
berlin_data['Stichtag'] = berlin_data.Stichtag.str.replace("\d+", "")#default value might change in future versions...
berlin_data['Stichtag'] = berlin_data.Stichtag.str.replace(" ", "")
for berlin_location in berlin_data.Stichtag:
    berlin_data.loc[berlin_data.Stichtag == berlin_location, "Stichtag"] = "SK Berlin "+berlin_location
print(berlin_data)

                                Stichtag     2005     2006     2007     2008  \
1                       SK Berlin Berlin  3395189  3404037  3416255  3431675   
2                        SK Berlin Mitte   322744   326422   328428   329592   
3     SK Berlin Friedrichshain-Kreuzberg   261811   265857   267919   270572   
4                       SK Berlin Pankow   353910   358210   363601   366899   
5   SK Berlin Charlottenburg-Wilmersdorf   315080   315557   316887   318208   
6                      SK Berlin Spandau   225042   224304   223652   223813   
7          SK Berlin Steglitz-Zehlendorf   288670   288575   290305   291871   
8         SK Berlin Tempelhof-Schöneberg   333163   332066   331790   333110   
9                     SK Berlin Neukölln   305900   305458   306713   309892   
10            SK Berlin Treptow-Köpenick   235336   236524   237777   239405   
11         SK Berlin Marzahn-Hellersdorf   250413   249881   249351   248150   
12                 SK Berlin Lichtenberg

/tmp/ipykernel_3344138/119497657.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  berlin_data['Stichtag'] = berlin_data.Stichtag.str.replace("\d+", "")#default value might change in future versions...


## Combine locations that have been reformed

In [2]:
all_years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012",
             "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

reformation_dict={"Region Hannover Landkreis":[["Hannover kreisfreie Stadt (bis 31.10.2001)",
                                            "Hannover Landkreis (bis 31.10.2001)"],["2000"]],
            "Göttingen Landkreis":[["Göttingen Landkreis (bis 31.10.2016)",
                                    "Osterode am Harz Landkreis (bis 31.10.2016)"],all_years[:16]],
            "Städteregion Aachen Landkreis":[["Aachen Landkreis (bis 20.10.2009)",
                                              "Aachen kreisfreie Stadt (bis 20.10.2009)"],all_years[:9]],
             # Kreisreform Mecklenburg Vorpommern
             "Mecklenburgische Seenplatte Landkreis":[["Neubrandenburg kreisfreie Stadt (bis 03.09.2011)",
                                                       "Demmin Landkreis (bis 03.09.2011)",
                                                       "Mecklenburg-Strelitz Landkreis (bis 03.09.2011)",
                                                       "Müritz Landkreis (bis 03.09.2011)"],all_years[:11]],
             "Rostock Landkreis":[["Bad Doberan Landkreis (bis 03.09.2011)",
                                   "Güstrow Landkreis (bis 03.09.2011)"],all_years[:11]],
             "Vorpommern-Rügen Landkreis":[["Nordvorpommern Landkreis (bis 03.09.2011)", 
                                            "Rügen Landkreis (bis 03.09.2011)", 
                                            "Stralsund kreisfreie Stadt (bis 03.09.2011)"],all_years[:11]],
             "Nordwestmecklenburg Landkreis":[["Nordwestmecklenburg Landkreis (bis 03.09.2011)",
                                               "Wismar kreisfreie Stadt (bis 03.09.2011)"],all_years[:11]],
             "Vorpommern-Greifswald Landkreis":[["Greifswald kreisfreie Stadt (bis 03.09.2011)",
                                                 "Uecker-Randow Landkreis (bis 03.09.2011)", 
                                                 "Ostvorpommern Landkreis (bis 03.09.2011)"],all_years[:11]],
             "Ludwigslust-Parchim Landkreis":[["Parchim Landkreis (bis 03.09.2011)", 
                                               "Ludwigslust Landkreis (bis 03.09.2011)"],all_years[:11]],
             # Kreisreform Sachsen
             "Chemnitz kreisfreie Stadt":[["Chemnitz kreisfreie Stadt (bis 31.07.2008)"], all_years[:8]],
             "Erzgebirgskreis":[["Annaberg Landkreis (bis 31.07.2008)",
                                 "Aue-Schwarzenberg Landkreis (bis 31.07.2008)",
                                 "Stollberg Landkreis (bis 31.07.2008)",
                                 "Mittlerer Erzgebirgskreis (bis 31.07.2008)"], all_years[:8]],
             "Mittelsachsen Landkreis":[["Freiberg Landkreis (bis 31.07.2008)",
                                         "Döbeln Landkreis (bis 31.07.2008)",
                                         "Mittweida Landkreis (bis 31.07.2008)"], all_years[:8]],
             "Vogtlandkreis":[["Plauen kreisfreie Stadt (bis 31.07.2008)",
                               "Vogtlandkreis (bis 31.07.2008)"], all_years[:8]],
             "Zwickau Landkreis":[["Zwickau kreisfreie Stadt (bis 31.07.2008)",
                                   "Zwickauer Land Landkreis (bis 31.07.2008)",
                                  "Chemnitzer Land Landkreis (bis 31.07.2008)"], all_years[:8]],
             "Dresden kreisfreie Stadt":[["Dresden kreisfreie Stadt (bis 31.07.2008)"], all_years[:8]],
             "Bautzen Landkreis":[["Hoyerswerda kreisfreie Stadt (bis 31.07.2008)",
                                   "Bautzen Landkreis (bis 31.07.2008)",
                                  "Kamenz Landkreis (bis 31.07.2008)"], all_years[:8]],
             "Görlitz Landkreis":[["Görlitz kreisfreie Stadt (bis 31.07.2008)",
                                   "Löbau-Zittau Landkreis (bis 31.07.2008)",
                                   "Niederschlesischer Oberlausitzkreis (b.31.07.2008)"], all_years[:8]],
             "Meißen Landkreis":[["Meißen Landkreis (bis 31.07.2008)",
                                  "Riesa-Großenhain Landkreis (bis 31.07.2008)"], all_years[:8]],
             "Sächsische Schweiz-Osterzgebirge Landkreis":[["Sächsische Schweiz Landkreis (bis 31.07.2008)",
                                                            "Weißeritzkreis (bis 31.07.2008)"], all_years[:8]],
             "Leipzig kreisfreie Stadt":[["Leipzig kreisfreie Stadt (bis 31.07.2008)"], all_years[:8]],
             "Leipzig Landkreis":[["Leipziger Land Landkreis (bis 31.07.2008)",
                                   "Muldentalkreis (bis 31.07.2008)"], all_years[:8]],
             "Nordsachsen Landkreis":[["Torgau-Oschatz Landkreis (bis 31.07.2008)",
                                       "Delitzsch Landkreis (bis 31.07.2008)"], all_years[:8]],
             # Kreisreform Sachsen Anhalt
             "Dessau-Roßlau kreisfreie Stadt":[["Dessau kreisfreie Stadt (bis 30.06.2007)"],all_years[:7]],
             "Halle (Saale) kreisfreie Stadt":[["Halle (Saale) kreisfreie Stadt (bis 30.06.2007)"],all_years[:7]],
             "Magdeburg kreisfreie Stadt":[["Magdeburg kreisfreie Stadt (bis 30.06.2007)"],all_years[:7]],
             "Altmarkkreis Salzwedel":[["Altmarkkreis Salzwedel (bis 30.06.2007)"],all_years[:7]],
             "Anhalt-Bitterfeld Landkreis":[["Anhalt-Zerbst Landkreis (bis 30.06.2007)",
                                             "Bitterfeld Landkreis (bis 30.06.2007)",
                                             "Köthen Landkreis (bis 30.06.2007)"],all_years[:7]],
             "Börde Landkreis":[["Bördekreis (bis 30.06.2007)",
                                 "Ohrekreis (bis 30.06.2007)"],all_years[:7]],
             "Burgenlandkreis":[["Weißenfels Landkreis (bis 30.06.2007)",
                                 "Burgenlandkreis (bis 30.06.2007)"],all_years[:7]],
             "Harz Landkreis":[["Wernigerode Landkreis (bis 30.06.2007)",
                                "Halberstadt Landkreis (bis 30.06.2007)",
                                "Quedlinburg Landkreis (bis 30.06.2007)"],all_years[:7]],
             "Jerichower Land Landkreis":[["Jerichower Land Landkreis (bis 30.06.2007)", ""],all_years[:7]],
             "Mansfeld-Südharz Landkreis":[["Mansfelder Land Landkreis (bis 30.06.2007)",
                                            "Sangerhausen Landkreis (bis 30.06.2007)"],all_years[:7]],
             "Saalekreis":[["Merseburg-Querfurt Landkreis (bis 30.06.2007)",
                            "Saalkreis (bis 30.06.2007)"],all_years[:7]],
             "Salzlandkreis":[["Schönebeck Landkreis (bis 30.06.2007)",
                               "Aschersleben-Staßfurt Landkreis (bis 30.06.2007)",
                               "Bernburg Landkreis (bis 30.06.2007)"],all_years[:7]],
             "Stendal Landkreis":[["Stendal Landkreis (bis 30.06.2007)"],all_years[:7]],
             "Wittenberg Landkreis":[["Wittenberg Landkreis (bis 30.06.2007)"],all_years[:7]],
             "Wartburgkreis":[["Wartburgkreis", "Eisenach kreisfreie Stadt (bis 30.06.2021)"], ["2021"]]
            }

# Iterate through the dict and the given years, take the sum of the reformed regions to fill the empty population numbers
for k,v in zip(reformation_dict.keys(), reformation_dict.values()):
    # handle each year seperately
    for year in v[1]:
        population_df.loc[population_df.Stichtag == k, year] = np.sum(population_df.loc[population_df.Stichtag.isin(v[0]), year].values)
# drop the rows with numbers in the Stichtag column -> leading to a drop of the rows with (bis ...) in them  
population_df.drop(population_df.loc[population_df.Stichtag.str.contains("\d+")].index, inplace=True)
display(population_df.loc[population_df.Stichtag.isin(reformation_dict.keys())])
print(len(list(population_df.Stichtag)))

,Kreise,Stichtag,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
27,3159,Göttingen Landkreis,350847,349808,348949,347660,346317,344007,342471,341560,...,323233,322616,324013,329538,327065,328036,328074,326041,323900,323661
29,3241,Region Hannover Landkreis,1118091,1122038,1125878,1126724,1128336,1128543,1128772,1130039,...,1112675,1119526,1128037,1144481,1148700,1152675,1157624,1157115,1155330,1157541
86,5334,Städteregion Aachen Landkreis,550966,553714,556963,566224,568037,568475,568863,568959,...,542833,545067,547661,553922,552472,554068,555465,557026,556631,556673
367,13071,Mecklenburgische Seenplatte Landkreis,325502,321317,317029,313320,309497,305939,302125,297662,...,264261,262412,261733,262517,261816,260574,259130,258074,258057,257525
368,13072,Rostock Landkreis,230656,229924,229061,228196,227193,225616,223780,221439,...,210732,210555,211878,213473,213945,214635,215113,215794,217072,217796
369,13073,Vorpommern-Rügen Landkreis,254927,252094,249375,247148,244858,242179,239653,237191,...,223718,223109,223470,224820,224971,225123,224684,224702,225383,225900
370,13074,Nordwestmecklenburg Landkreis,168357,167697,167129,166533,166085,165704,164544,163689,...,155801,155265,155424,156270,156825,156993,156729,157322,157975,158449
371,13075,Vorpommern-Greifswald Landkreis,253940,250615,247236,245042,242519,240722,238915,237375,...,239291,238185,237697,238358,237374,237066,236697,235623,235773,235451
372,13076,Ludwigslust-Parchim Landkreis,240548,239288,237793,235990,233398,231162,228589,226058,...,212373,211965,212631,214113,212562,212522,212618,211779,211844,211899
402,14511,Chemnitz kreisfreie Stadt,259246,255798,252618,249922,248365,246587,245700,244951,...,241210,242022,243521,248645,246353,246855,247237,246334,244401,243105


400


## Names

In [15]:
manually_defined_dict = {'SK Oldenburg (Oldenburg)': ['SK Oldenburg'], 'Oldenburg': ['LK Oldenburg'], 'Osnabrück': ['LK Osnabrück'],
                         'SK Mülheim an der Ruhr': ['SK Mülheim a.d.Ruhr'], 'Städteregion Aachen': ['StädteRegion Aachen'],
                         'SK Offenbach am Main': ['SK Offenbach'], 'Offenbach': ['LK Offenbach'], 'Kassel': ['LK Kassel'],
                         'Altenkirchen (Westerwald)': ['LK Altenkirchen'], 'Eifelkreis Bitburg-Prüm': ['LK Bitburg-Prüm'],
                         'SK Frankenthal (Pfalz)': ['SK Frankenthal'], 'SK Landau in der Pfalz': ['SK Landau i.d.Pfalz'],
                         'SK Ludwigshafen am Rhein': ['SK Ludwigshafen'], 'SK Neustadt an der Weinstraße': ['SK Neustadt a.d.Weinstraße'],
                         'Kaiserslautern': ['LK Kaiserslautern'], 'Heilbronn': ['LK Heilbronn'], 'Karlsruhe': ['LK Karlsruhe'],
                         'SK Freiburg im Breisgau': ['SK Freiburg i.Breisgau'], 'Landsberg am Lech': ['LK Landsberg a.Lech'],
                         'Mühldorf am Inn': ['LK Mühldorf a.Inn'], 'München': ['LK München'], 'Pfaffenhofen an der Ilm': ['LK Pfaffenhofen a.d.Ilm'],
                         'Rosenheim': ['LK Rosenheim'], 'Landshut': ['LK Landshut'], 'Passau': ['LK Passau'],
                         'Regen': ['LK Regen'], 'SK Weiden in der Oberpfalz': ['SK Weiden i.d.OPf.'], 'Neumarkt in der Oberpfalz': ['LK Neumarkt i.d.OPf.'],
                         'Neustadt an der Waldnaab': ['LK Neustadt a.d.Waldnaab'], 'Regensburg': ['LK Regensburg'], 'Bamberg': ['LK Bamberg'],
                         'Bayreuth': ['LK Bayreuth'], 'Coburg': ['LK Coburg'], 'Hof': ['LK Hof'], 
                         'Wunsiedel im Fichtelgebirge': ['LK Wunsiedel i.Fichtelgebirge'], 'Ansbach': ['LK Ansbach'], 'Fürth': ['LK Fürth'],
                         'Neustadt an der Aisch-Bad Windsheim': ['LK Neustadt a.d.Aisch-Bad Windsheim'], 'Aschaffenburg': ['LK Aschaffenburg'],
                         'Schweinfurt': ['LK Schweinfurt'], 'Würzburg': ['LK Würzburg'], 'SK Kempten (Allgäu)': ['SK Kempten'],
                         'Augsburg': ['LK Augsburg'], 'Dillingen an der Donau': ['LK Dillingen a.d.Donau'], 'Lindau (Bodensee)': ['LK Lindau'],
                         'Regionalverband Saarbrücken': ['LK Stadtverband Saarbrücken'], 'SK Brandenburg an der Havel': ['SK Brandenburg a.d.Havel'],
                         'Rostock': ['LK Rostock'], 'Leipzig': ['LK Leipzig'], 'SK Halle (Saale)': ['SK Halle'],}

In [16]:
population_locations = population_df.Stichtag.unique()
influenza_locations = influenza_df.location.unique()
renaming_dict = dict()
population_df["Location"] = population_df["Stichtag"].copy()
population_df.loc[population_df.Location.str.contains(" kreisfreie Stadt"), "Location"] = population_df.loc[population_df.Location.str.contains(" kreisfreie Stadt"),
                                                                                                           "Location"].apply(lambda s: 'SK '+s)
population_df['Location'] = population_df.Location.str.replace(" kreisfreie Stadt" , "")

for lk_indicator in [" Landkreis"]:#, "kreis", "-Kreis", " Kreis"]:
    #population_df.loc[population_df.Location.str.contains(lk_indicator), "Location"] = population_df.loc[population_df.Location.str.contains(lk_indicator),
     #                                                                                                      "Location"].apply(lambda s: 'LK '+s)
    population_df['Location'] = population_df.Location.str.replace(lk_indicator , "")
#print(population_df.loc[~(population_df.Location.str.contains("LK") | population_df.Location.str.contains("SK"))])
#print(population_df)
unwanted_strings = [f"{i}" for i in range(0,10)]+[".", "  (bis )", " (bis )"]
for unwanted_string in unwanted_strings:
    population_df.Location = population_df.Location.apply(lambda s: s.replace(unwanted_string, "") if unwanted_string in s else s)
    
renaming_dict = {}
for location in population_df.Location:
    renaming_dict[location] = []
    for location2 in influenza_locations:
        if location in location2 :
            renaming_dict[location].append(location2)
    # overwrite some locations with manually defined matches
    if location in manually_defined_dict.keys():
        renaming_dict[location] = manually_defined_dict[location]
            
# save the entries of the renaming dict where we have no match or at least 2 matches
not_matching_dict = {k:v for k, v in zip(renaming_dict.keys(), renaming_dict.values()) if ((len(v)==0) | (len(v)>1))}
# also save the entries where we have exactly one match (as it's supposed to be)
correctly_matched_dict = {k:v for k, v in zip(renaming_dict.keys(), renaming_dict.values()) if len(v) == 1}

m = [loc for loc in influenza_locations if loc not in [loc2[0] for loc2 in correctly_matched_dict.values()]]

print(m)

['SK Berlin Charlottenburg-Wilmersdorf', 'SK Berlin Friedrichshain-Kreuzberg', 'SK Berlin Lichtenberg', 'SK Berlin Marzahn-Hellersdorf', 'SK Berlin Mitte', 'SK Berlin Neukölln', 'SK Berlin Pankow', 'SK Berlin Reinickendorf', 'SK Berlin Spandau', 'SK Berlin Steglitz-Zehlendorf', 'SK Berlin Tempelhof-Schöneberg', 'SK Berlin Treptow-Köpenick']


## Add the berlin data to the remaining regional populational data

## Renamed locations

In [10]:
locations_with_nan = dict()
for i in range(2000,2020):
    locations_with_nan[i] = list(population_df.loc[population_df[f"{i}"] == "-","Stichtag"])
#print(locations_with_nan)
old_locations = [location for location in list(population_df["Stichtag"]) if "(bis" in location]
print(old_locations)

['Göttingen Landkreis (bis 31.10.2016)', 'Osterode am Harz Landkreis (bis 31.10.2016)', 'Hannover kreisfreie Stadt (bis 31.10.2001)', 'Hannover Landkreis (bis 31.10.2001)', 'Aachen kreisfreie Stadt (bis 20.10.2009)', 'Aachen Landkreis (bis 20.10.2009)', 'Greifswald kreisfreie Stadt (bis 03.09.2011)', 'Neubrandenburg kreisfreie Stadt (bis 03.09.2011)', 'Stralsund kreisfreie Stadt (bis 03.09.2011)', 'Wismar kreisfreie Stadt (bis 03.09.2011)', 'Bad Doberan Landkreis (bis 03.09.2011)', 'Demmin Landkreis (bis 03.09.2011)', 'Güstrow Landkreis (bis 03.09.2011)', 'Ludwigslust Landkreis (bis 03.09.2011)', 'Mecklenburg-Strelitz Landkreis (bis 03.09.2011)', 'Müritz Landkreis (bis 03.09.2011)', 'Nordvorpommern Landkreis (bis 03.09.2011)', 'Nordwestmecklenburg Landkreis (bis 03.09.2011)', 'Ostvorpommern Landkreis (bis 03.09.2011)', 'Parchim Landkreis (bis 03.09.2011)', 'Rügen Landkreis (bis 03.09.2011)', 'Uecker-Randow Landkreis (bis 03.09.2011)', 'Chemnitz kreisfreie Stadt (bis 31.07.2008)', 'Plau

In [12]:
luc = "bis 30.06.2007"
print(population_df.loc[population_df.Stichtag.str.contains(luc)]["Stichtag"])

429            Dessau kreisfreie Stadt (bis 30.06.2007)
430            Anhalt-Zerbst Landkreis (bis 30.06.2007)
431                 Bernburg Landkreis (bis 30.06.2007)
432               Bitterfeld Landkreis (bis 30.06.2007)
433                   Köthen Landkreis (bis 30.06.2007)
434               Wittenberg Landkreis (bis 30.06.2007)
435     Halle (Saale) kreisfreie Stadt (bis 30.06.2007)
436                    Burgenlandkreis (bis 30.06.2007)
437          Mansfelder Land Landkreis (bis 30.06.2007)
438       Merseburg-Querfurt Landkreis (bis 30.06.2007)
439                          Saalkreis (bis 30.06.2007)
440             Sangerhausen Landkreis (bis 30.06.2007)
441               Weißenfels Landkreis (bis 30.06.2007)
442         Magdeburg kreisfreie Stadt (bis 30.06.2007)
443    Aschersleben-Staßfurt Landkreis (bis 30.06.2007)
444                         Bördekreis (bis 30.06.2007)
445              Halberstadt Landkreis (bis 30.06.2007)
446          Jerichower Land Landkreis (bis 30.0